In [88]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
import xgboost as xgb
import catboost as cat
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import  train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.pipeline import Pipeline
import numpy as np
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.feature_selection import RFECV, SelectFromModel


In [89]:
df = pd.read_csv(r'C:\Users\LENOVO\.spyder-py3\exercise\playground-series-s5e5\train.csv')
df_test = pd.read_csv(r'C:\Users\LENOVO\.spyder-py3\exercise\playground-series-s5e5\test.csv')


In [90]:
print(df.info())
print(df.isnull().sum())



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 750000 entries, 0 to 749999
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   id          750000 non-null  int64  
 1   Sex         750000 non-null  object 
 2   Age         750000 non-null  int64  
 3   Height      750000 non-null  float64
 4   Weight      750000 non-null  int64  
 5   Duration    750000 non-null  int64  
 6   Heart_Rate  750000 non-null  float64
 7   Body_Temp   750000 non-null  float64
 8   Calories    750000 non-null  int64  
dtypes: float64(3), int64(5), object(1)
memory usage: 51.5+ MB
None
id            0
Sex           0
Age           0
Height        0
Weight        0
Duration      0
Heart_Rate    0
Body_Temp     0
Calories      0
dtype: int64


In [91]:
df['BMI'] = df['Weight'] / (df['Height']/100)**2
df_test['BMI'] = df_test['Weight'] / (df_test['Height']/100)**2

In [92]:
def high_classify(df):
    if df['Sex'] == 'male':
        if df['Height'] < 160.0:
            return 'Short'
        elif 160.0 <= df['Height'] < 180.0:
            return 'Average'
        elif df['Height'] >= 180.0:
            return 'High'
        else:
            return 'Others'
    else:
        if df['Height'] < 145.0:
            return 'Short'
        elif 145.0 <= df['Height'] < 170.0:
            return 'Average'
        elif df['Height'] >= 170:
            return 'High'
        else:
            return 'Others'

In [93]:
df['Height_classify'] = df.apply(high_classify, axis = 1)
df_test['Height_classify'] = df_test.apply(high_classify, axis = 1)

In [94]:
df['Height_classify'].value_counts()

Height_classify
High       375418
Average    372901
Short        1681
Name: count, dtype: int64

In [95]:
def bmi_classify(df):
    bmi = df['BMI']
    if bmi < 18.5 or bmi >= 30:
        return 'Abnormal'
    elif bmi < 25:
        return 'Normal'
    else:
        return 'Overweight'


In [96]:
df['BMI_classify'] = df.apply(bmi_classify, axis = 1)
df_test['BMI_classify'] = df_test.apply(bmi_classify, axis = 1)


In [97]:
df['BMI_classify'].value_counts()

BMI_classify
Normal        489058
Overweight    260735
Abnormal         207
Name: count, dtype: int64

In [98]:
def age_classify(df):
    if df['Age'] < 35:
        return 'adult'
    elif 35 <= df['Age'] < 60:
        return 'middle_age'
    elif 60 <= df['Age'] < 80:
        return 'elderly'
    else:
        return 'Others'

In [99]:
df['Age_classify'] = df.apply(age_classify, axis = 1)
df_test['Age_classify'] = df_test.apply(age_classify, axis = 1)


In [100]:
high = ['Short','Average','High','Others']
bmi = ['Abnormal','Normal', 'Overweight']
age = ['adult','middle_age','elderly','Others']

columns_to_encode = ['Height_classify', 'BMI_classify', 'Age_classify']
df[columns_to_encode] = df[columns_to_encode].astype(str)
df_test[columns_to_encode] = df_test[columns_to_encode].astype(str)

ord_encoded = OrdinalEncoder(categories=[high, bmi, age], handle_unknown='use_encoded_value', unknown_value=-1)

df[columns_to_encode] = ord_encoded.fit_transform(df[columns_to_encode]).astype(int)
df_test[columns_to_encode] = ord_encoded.transform(df_test[columns_to_encode]).astype(int)


In [101]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 750000 entries, 0 to 749999
Data columns (total 13 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   id               750000 non-null  int64  
 1   Sex              750000 non-null  object 
 2   Age              750000 non-null  int64  
 3   Height           750000 non-null  float64
 4   Weight           750000 non-null  int64  
 5   Duration         750000 non-null  int64  
 6   Heart_Rate       750000 non-null  float64
 7   Body_Temp        750000 non-null  float64
 8   Calories         750000 non-null  int64  
 9   BMI              750000 non-null  float64
 10  Height_classify  750000 non-null  int32  
 11  BMI_classify     750000 non-null  int32  
 12  Age_classify     750000 non-null  int32  
dtypes: float64(4), int32(3), int64(5), object(1)
memory usage: 65.8+ MB


In [102]:
df

,id,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Calories,BMI,Height_classify,BMI_classify,Age_classify
0,0,male,36,189.0,82,26,101.0,41.0,150,22.955684,2,1,1
1,1,female,64,163.0,60,8,85.0,39.7,34,22.582709,1,1,2
2,2,female,51,161.0,64,7,84.0,39.8,29,24.690405,1,1,1
3,3,male,20,192.0,90,25,105.0,40.7,140,24.414062,2,1,0
4,4,female,38,166.0,61,25,102.0,40.6,146,22.136740,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
749995,749995,male,28,193.0,97,30,114.0,40.9,230,26.040968,2,2,0
749996,749996,female,64,165.0,63,18,92.0,40.5,96,23.140496,1,1,2
749997,749997,male,60,162.0,67,29,113.0,40.9,221,25.529645,1,2,2
749998,749998,male,45,182.0,91,17,102.0,40.3,109,27.472527,2,2,1


In [103]:
df['Sex'] = df['Sex'].map({
    'male': 0,
    'female': 1
})

In [104]:
def combine_cat_feature(df, cols):
    combined = df[cols[0]].astype(str)
    for i in range(1, len(cols)):
        combined = combined.str.cat(df[cols[i]].astype(str), sep='')
    return combined

In [105]:
df['Age_Sex'] = combine_cat_feature(df,['Age_classify','Sex'])
df['Age_BMI'] = combine_cat_feature(df, ['Age_classify','BMI_classify'])
df['Age_Heigh'] = combine_cat_feature(df, ['Age_classify','Height_classify'])
df['BMI_Heigh'] = combine_cat_feature(df, ['BMI_classify','Height_classify'])
df['BMI_Sex'] = combine_cat_feature(df, ['BMI_classify','Sex'])
df['Heigh_Sex'] = combine_cat_feature(df, ['Height_classify','Sex'])

# combine 3 feature
df['Age_Sex_BMI'] = combine_cat_feature(df,['Age_classify','Sex','BMI_classify'])
df['Age_Sex_Heigh'] = combine_cat_feature(df,['Age_classify','Sex','Height_classify'])
df['Age_BMI_Heigh'] = combine_cat_feature(df, ['Age_classify','BMI_classify','Height_classify'])
df['BMI_Sex_Heigh'] = combine_cat_feature(df, ['BMI_classify','Sex','Height_classify'])


In [106]:
df_test['Age_Sex'] = combine_cat_feature(df_test,['Age_classify','Sex'])
df_test['Age_BMI'] = combine_cat_feature(df_test, ['Age_classify','BMI_classify'])
df_test['Age_Heigh'] = combine_cat_feature(df_test, ['Age_classify','Height_classify'])
df_test['BMI_Heigh'] = combine_cat_feature(df_test, ['BMI_classify','Height_classify'])
df_test['BMI_Sex'] = combine_cat_feature(df_test, ['BMI_classify','Sex'])
df_test['Heigh_Sex'] = combine_cat_feature(df_test, ['Height_classify','Sex'])

df_test['Age_Sex_BMI'] = combine_cat_feature(df_test,['Age_classify','Sex','BMI_classify'])
df_test['Age_Sex_Heigh'] = combine_cat_feature(df_test,['Age_classify','Sex','Height_classify'])
df_test['Age_BMI_Heigh'] = combine_cat_feature(df_test, ['Age_classify','BMI_classify','Height_classify'])
df_test['BMI_Sex_Heigh'] = combine_cat_feature(df_test, ['BMI_classify','Sex','Height_classify'])

In [107]:
cols = df.select_dtypes('object')
for col in cols:
    df[col] = df[col].astype(int)

cols_test = df_test.select_dtypes('object')
for col_test in cols_test:
    df_test[col_test] = df_test[col_test].astype(int)

In [108]:
from sklearn.metrics import make_scorer
import numpy as np

def rmsle(y_true, y_pred):
    y_true = np.maximum(0, y_true)
    y_pred = np.maximum(0, y_pred)
    return np.sqrt(np.mean((np.log1p(y_pred) - np.log1p(y_true))**2))

rmsle_scorer = make_scorer(rmsle, greater_is_better=False)


In [24]:
X= df.drop('Calories', axis=1)
y= df['Calories']

In [ ]:
import optuna #Hypertunning


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define categorical columns
categorical_cols = ['Sex', 'BMI_classify', 'Age_classify']

# Define RMSLE function
def rmsle(y_true, y_pred):
    y_pred = np.clip(y_pred, 0, None)
    return np.sqrt(np.mean(np.square(np.log1p(y_true) - np.log1p(y_pred))))

# Define the objective function for Optuna with RMSLE
def objective(trial):
    params = {
        'iterations': trial.suggest_int('iterations', 1000, 5000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'depth': trial.suggest_int('depth', 6, 16),
        'loss_function': 'RMSE',  # CatBoost still uses RMSE internally
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1.0, 10.0),
        'random_seed': 42,
        'eval_metric': 'RMSE',
        'early_stopping_rounds': 200,
        'task_type': 'GPU',
        'verbose': 0,
        'cat_features': categorical_cols
    }
    
    pipeline = Pipeline([
        ('model', CatBoostRegressor(**params))
    ])
    
    pipeline.fit(X_train, y_train, model__eval_set=(X_test, y_test))
    y_pred = pipeline.predict(X_test)
    rmsle_score = rmsle(y_test, y_pred)  # Optimize for RMSLE
    return rmsle_score

# Create and optimize study for RMSLE
study = optuna.create_study(direction='minimize')  # Minimize RMSLE
study.optimize(objective, n_trials=50)
print("Best parameters for RMSLE:", study.best_params)
print("Best RMSLE:", study.best_value)

[I 2025-05-19 00:46:41,905] A new study created in memory with name: no-name-27f292fa-b4e9-423c-a97d-db342808a1e6
[I 2025-05-19 00:48:57,940] Trial 0 finished with value: 0.06142202206857082 and parameters: {'iterations': 2441, 'learning_rate': 0.045680624588363264, 'depth': 15, 'l2_leaf_reg': 4.025777350323823}. Best is trial 0 with value: 0.06142202206857082.
[I 2025-05-19 00:49:50,248] Trial 1 finished with value: 0.061508426558371576 and parameters: {'iterations': 3382, 'learning_rate': 0.06374996066320485, 'depth': 13, 'l2_leaf_reg': 6.4114960093912545}. Best is trial 0 with value: 0.06142202206857082.
[I 2025-05-19 00:50:32,018] Trial 2 finished with value: 0.061516585120256415 and parameters: {'iterations': 4311, 'learning_rate': 0.1460301810509182, 'depth': 14, 'l2_leaf_reg': 2.240827317977971}. Best is trial 0 with value: 0.06142202206857082.
[I 2025-05-19 00:51:55,506] Trial 3 finished with value: 0.06193063687244312 and parameters: {'iterations': 4761, 'learning_rate': 0.014

Best parameters for RMSLE: {'iterations': 3773, 'learning_rate': 0.012003415305757718, 'depth': 13, 'l2_leaf_reg': 2.8038375623387}
Best RMSLE: 0.06070331880713431


In [48]:
X_sel = df.drop('Calories', axis=1)
Y = df['Calories']
pipe = Pipeline([
    ('scale', StandardScaler()),
    ('model', cat.CatBoostRegressor())
])

In [49]:
# Using optuna
grid = {
    'model__iterations': [3773],
    'model__learning_rate': [0.012003415305757718],
    'model__depth': [13],
    'model__loss_function': ['RMSE'],
    'model__l2_leaf_reg': [2.8038375623387],
    'model__random_seed': [42],
    'model__eval_metric': ['RMSE'],
    'model__early_stopping_rounds': [200],
    'model__verbose': [100],
    'model__task_type': ['GPU'],
}


In [50]:
grid_search = GridSearchCV(pipe, grid, scoring=rmsle_scorer, cv=10,  n_jobs=-1)
grid_search.fit(X_sel, Y)
print(grid_search.best_params_)

0:	learn: 61.6674832	total: 21.8ms	remaining: 1m 22s
100:	learn: 19.5463126	total: 2.4s	remaining: 1m 27s
200:	learn: 7.2123150	total: 4.81s	remaining: 1m 25s
300:	learn: 4.2190334	total: 7.26s	remaining: 1m 23s
400:	learn: 3.6834837	total: 9.65s	remaining: 1m 21s
500:	learn: 3.5574612	total: 12s	remaining: 1m 18s
600:	learn: 3.5016542	total: 14.4s	remaining: 1m 15s
700:	learn: 3.4647301	total: 16.7s	remaining: 1m 13s
800:	learn: 3.4366367	total: 19.3s	remaining: 1m 11s
900:	learn: 3.4126245	total: 21.8s	remaining: 1m 9s
1000:	learn: 3.3919371	total: 24.3s	remaining: 1m 7s
1100:	learn: 3.3742492	total: 27s	remaining: 1m 5s
1200:	learn: 3.3575055	total: 29.4s	remaining: 1m 3s
1300:	learn: 3.3423054	total: 31.9s	remaining: 1m
1400:	learn: 3.3275508	total: 34.3s	remaining: 58s
1500:	learn: 3.3139034	total: 36.6s	remaining: 55.3s
1600:	learn: 3.3006917	total: 38.9s	remaining: 52.8s
1700:	learn: 3.2882056	total: 41.4s	remaining: 50.5s
1800:	learn: 3.2755607	total: 44s	remaining: 48.2s
1900:

In [51]:
feature_importances = grid_search.best_estimator_.named_steps['model'].feature_importances_

important_df = pd.DataFrame({
    'Feature': X_sel.columns,
    'Importance': feature_importances
}).sort_values(by='Importance', ascending=False)

print(important_df)

            Feature  Importance
5          Duration   50.678541
6        Heart_Rate   23.237805
2               Age    9.282547
1               Sex    5.076523
7         Body_Temp    3.010592
4            Weight    1.696092
12          Age_Sex    1.386821
18      Age_Sex_BMI    1.019676
13          Age_BMI    0.922881
19    Age_Sex_Heigh    0.897864
20    Age_BMI_Heigh    0.727394
3            Height    0.577741
14        Age_Heigh    0.523405
11     Age_classify    0.372053
0                id    0.178857
8               BMI    0.142607
17        Heigh_Sex    0.133303
9   Height_classify    0.070970
21    BMI_Sex_Heigh    0.036301
15        BMI_Heigh    0.014732
16          BMI_Sex    0.012166
10     BMI_classify    0.001130


In [52]:
# Predictions for Grid_search (already computed)
y_pred_1 = grid_search.best_estimator_.predict(X_sel)  # As per your code
rmsle_grid = np.sqrt(np.mean((np.log1p(Y) - np.log1p(y_pred_1))**2))
print('RMSLE for Grid_search:', rmsle_grid
)

print("RMSLE for CatBoostRegressor:" , study.best_value)


RMSLE for Grid_search: 0.056743427659777024
RMSLE for CatBoostRegressor: 0.06070331880713431


In [109]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 750000 entries, 0 to 749999
Data columns (total 23 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   id               750000 non-null  int64  
 1   Sex              750000 non-null  int64  
 2   Age              750000 non-null  int64  
 3   Height           750000 non-null  float64
 4   Weight           750000 non-null  int64  
 5   Duration         750000 non-null  int64  
 6   Heart_Rate       750000 non-null  float64
 7   Body_Temp        750000 non-null  float64
 8   Calories         750000 non-null  int64  
 9   BMI              750000 non-null  float64
 10  Height_classify  750000 non-null  int32  
 11  BMI_classify     750000 non-null  int32  
 12  Age_classify     750000 non-null  int32  
 13  Age_Sex          750000 non-null  int32  
 14  Age_BMI          750000 non-null  int32  
 15  Age_Heigh        750000 non-null  int32  
 16  BMI_Heigh        750000 non-null  int3

In [111]:
df_test.drop('Calories', axis=1)


,id,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,BMI,Height_classify,...,Age_Sex,Age_BMI,Age_Heigh,BMI_Heigh,BMI_Sex,Heigh_Sex,Age_Sex_BMI,Age_Sex_Heigh,Age_BMI_Heigh,BMI_Sex_Heigh
0,750000,1,45,177,81,7,87,39.8,25.854639,2,...,11,12,12,22,21,21,112,112,122,212
1,750001,1,26,200,97,20,101,40.5,24.250000,2,...,1,1,2,12,11,21,11,12,12,112
2,750002,0,29,188,85,16,102,40.4,24.049344,2,...,0,1,2,12,10,20,1,2,12,102
3,750003,0,39,172,73,20,107,40.6,24.675500,2,...,10,11,12,12,10,20,101,102,112,102
4,750004,0,30,173,67,16,94,40.5,22.386314,2,...,0,1,2,12,10,20,1,2,12,102
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249995,999995,0,56,159,62,6,85,39.4,24.524346,1,...,10,11,11,11,10,10,101,101,111,101
249996,999996,1,32,202,101,3,84,38.4,24.752475,2,...,1,1,2,12,11,21,11,12,12,112
249997,999997,0,31,164,64,14,98,40.1,23.795360,1,...,0,1,1,11,10,10,1,1,11,101
249998,999998,0,62,158,61,25,106,40.7,24.435187,1,...,20,21,21,11,10,10,201,201,211,101


In [112]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250000 entries, 0 to 249999
Data columns (total 23 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   id               250000 non-null  int64  
 1   Sex              250000 non-null  int64  
 2   Age              250000 non-null  int64  
 3   Height           250000 non-null  int64  
 4   Weight           250000 non-null  int64  
 5   Duration         250000 non-null  int64  
 6   Heart_Rate       250000 non-null  int64  
 7   Body_Temp        250000 non-null  float64
 8   Calories         250000 non-null  float64
 9   BMI              250000 non-null  float64
 10  Height_classify  250000 non-null  int32  
 11  BMI_classify     250000 non-null  int32  
 12  Age_classify     250000 non-null  int32  
 13  Age_Sex          250000 non-null  int32  
 14  Age_BMI          250000 non-null  int32  
 15  Age_Heigh        250000 non-null  int32  
 16  BMI_Heigh        250000 non-null  int3

In [114]:
# Drop 'Calories' column from df_test if it exists, to match training features
if 'Calories' in df_test.columns:
    df_test_features = df_test.drop('Calories', axis=1)
else:
    df_test_features = df_test

# Make predictions using the features only
df_test['Calories'] = grid_search.best_estimator_.predict(df_test_features)

print("Predictions added to df_test:")
print(df_test.head())

Predictions added to df_test:
       id  Sex  Age  Height  Weight  Duration  Heart_Rate  Body_Temp  \
0  750000    1   45     177      81         7          87       39.8   
1  750001    1   26     200      97        20         101       40.5   
2  750002    0   29     188      85        16         102       40.4   
3  750003    0   39     172      73        20         107       40.6   
4  750004    0   30     173      67        16          94       40.5   

     Calories        BMI  ...  Age_Sex  Age_BMI  Age_Heigh  BMI_Heigh  \
0   28.887246  25.854639  ...       11       12         12         22   
1  104.425039  24.250000  ...        1        1          2         12   
2   86.727761  24.049344  ...        0        1          2         12   
3  128.233556  24.675500  ...       10       11         12         12   
4   66.662023  22.386314  ...        0        1          2         12   

   BMI_Sex  Heigh_Sex  Age_Sex_BMI  Age_Sex_Heigh  Age_BMI_Heigh  \
0       21         21         

In [122]:
df_submission = df_test[['id', 'Calories']]
df_submission

,id,Calories
0,750000,28.887246
1,750001,104.425039
2,750002,86.727761
3,750003,128.233556
4,750004,66.662023
...,...,...
249995,999995,23.132539
249996,999996,13.160914
249997,999997,65.757375
249998,999998,182.722281


In [123]:
df_submission.to_csv('submission1.csv', index=False)